### Prep before starting script 
!!! Convert .czi to .lsm (LSM 5) on ZEN Black on the 880 for the pipeline to run. Save both files in the file folder!!!

Keep the .czi to create the dummy excel 
1. Open .czi file with FIJI BioFormats importer
2. Before you press open tick the metadata option 
3. Once the image opens go to the metadata window, Ctrl + A and Ctrl + C the contents
4. Ctrl + V the contents into an .txt file
5. Call the .txt file OME_"name of .czi file"




### Imports

In [3]:
import re
from datetime import datetime, timedelta
from openpyxl import Workbook

### Replace with your file paths

In [2]:
  
# metadata .txt file path
file_path_OME = r"Y:\Jacqui\Critical_Period\Example Analysis\WT_C1_Above\OME_WT_C1_Above_6dpf.txt"  

# Paths to the log files for ViSaGe and MCP systems.
# ViSage clicks 
log_v_path = r'Y:\Jacqui\Critical_Period\Example Analysis\click_log_ViSaGe_6dpf_05_Jul_25.txt'
# microscope PC clicks 
log_m_path = r'Y:\Jacqui\Critical_Period\Example Analysis\click_log_MPC_6dpf_05_Jul_25.txt'

# Specify the output file path where frame times will be saved.
output_file_path_fts = r"Y:\Jacqui\Critical_Period\Example Analysis\WT_C1_Above\frame_fimes.txt" #USE AS STORED VARIABLE: CHANGE LATER!!!!

# Final csv file path
output_file = r"Y:\Jacqui\Critical_Period\Example Analysis\WT_C1_Above\Time_Events_for_.csv"


### Part One: Find out which frame number coincides with the beginning of ViSaGe video
Aim: Determine which frames to match with a stimulus 

#### Step 1: Adjust time-series acquisition start to be on ViSaGe time 
##### Extract aquisition start time from microscope file metadata

In [4]:
def extract_and_adjust_acquisition_time(file_path):
    """
    Extracts the acquisition date and time from an OME metadata file,
    adds one hour to the time, and returns the adjusted timestamp.
    Also extracts the time in HH:MM:SS.ms format as mpc_time.
    """
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()

    match = re.search(r"<AcquisitionDate>(.*?)</AcquisitionDate>", content)
    
    if match:
        original_time_str = match.group(1)
        original_time = datetime.strptime(original_time_str, "%Y-%m-%dT%H:%M:%S.%f")
        adjusted_time = original_time + timedelta(hours=1)
        adjusted_time_str = adjusted_time.strftime("%Y-%m-%dT%H:%M:%S.%f")[:-3]
        print(f"Original: {original_time_str}")
        print(f"Adjusted (+1hr to get to BTS): {adjusted_time_str}")
        
        # Extract just the time part in HH:MM:SS.ms format
        mpc_time = adjusted_time_str.split("T")[1]
        print(f"mpc_time (HH:MM:SS.ms): {mpc_time}")
        return adjusted_time_str, mpc_time
    else:
        print("No <AcquisitionDate> tag found.")
        return None, None
 

adjusted_time_str, mpc_time = extract_and_adjust_acquisition_time(file_path_OME)
print(mpc_time) 

Original: 2025-06-05T16:17:09.605
Adjusted (+1hr to get to BTS): 2025-06-05T17:17:09.605
mpc_time (HH:MM:SS.ms): 17:17:09.605
17:17:09.605


#### Find time difference between computers and adjust acquisition time (MPC adjusted)
(MPC-ViSaGe = time difference, MPC - time difference = adjusted time to ViSaGe)


In [5]:

# Converts a timestamp (hours, minutes, seconds, milliseconds) into total milliseconds.
def timestamp_to_ms(h, m, s, ms):
    return int(h)*3600000 + int(m)*60000 + int(s)*1000 + int(ms)

# Converts milliseconds into a formatted string in MM:SS.ms format.
def ms_to_mmssms(ms):
    total_seconds = ms // 1000
    minutes = total_seconds // 60
    seconds = total_seconds % 60
    milliseconds = abs(ms) % 1000
    sign = "-" if ms < 0 else ""  # Handles negative time differences.
    return f"{sign}{minutes:02}:{seconds:02}.{milliseconds:03}"

# Converts milliseconds into a formatted string in HH:MM:SS.ms format.
def ms_to_hhmmssms(ms):
    total_seconds = ms // 1000
    hours = total_seconds // 3600
    minutes = (total_seconds % 3600) // 60
    seconds = total_seconds % 60
    milliseconds = abs(ms) % 1000
    sign = "-" if ms < 0 else ""  # Handles negative time differences.
    return f"{sign}{hours:02}:{minutes:02}:{seconds:02}.{milliseconds:03}"

# Calculates the average time difference between two log files (ViSaGe and MCP).
def find_time_diff(clicklogV, clicklogM):
    # Reads the content of the ViSaGe log file.
    with open(clicklogV, 'r') as logV:
        contentV = logV.read()
    # Reads the content of the MCP log file.
    with open(clicklogM, 'r') as logM:
        contentM = logM.read()

    # Regular expression to extract timestamps in HH:MM:SS.ms format.
    pattern = r'(\d{2}):(\d{2}):(\d{2})\.(\d{1,3})'
    # Converts timestamps from ViSaGe log into milliseconds.
    time_log_V = [timestamp_to_ms(*match) for match in re.findall(pattern, contentV)]
    # Converts timestamps from MCP log into milliseconds.
    time_log_M = [timestamp_to_ms(*match) for match in re.findall(pattern, contentM)]

    # Ensures both logs have the same number of timestamps for comparison.
    min_len = min(len(time_log_V), len(time_log_M))
    time_log_V = time_log_V[:min_len]
    time_log_M = time_log_M[:min_len]

    # Calculates the differences between corresponding timestamps in the two logs.
    differences = [m - v for v, m in zip(time_log_V, time_log_M)]
    # Computes the average time difference.
    average_diff = int(round(sum(differences) / len(differences)))

    # Prints the average time difference in milliseconds and MM:SS.ms format.
    print(f"Average time difference (MCP - ViSaGe): {average_diff} ms")
    print(f"Average time difference (MM:SS.ms): {ms_to_mmssms(average_diff)}")
    return average_diff

# Adjusts a given MCP timestamp by subtracting the average time difference.
def subtract_diff_from_time(mpc_time_str, avg_diff_ms):
    # Determines the format of the input timestamp (MM:SS.ms or HH:MM:SS.ms).
    if len(mpc_time_str.split(":")) == 2:
        # MM:SS.ms format
        m, s_ms = mpc_time_str.split(":")
        s, ms = s_ms.split(".")
        total_ms = int(m)*60000 + int(s)*1000 + int(ms)
    else:
        # HH:MM:SS.ms format
        h, m, s_ms = mpc_time_str.split(":")
        s, ms = s_ms.split(".")
        total_ms = int(h)*3600000 + int(m)*60000 + int(s)*1000 + int(ms)
    # Subtracts the average time difference from the total milliseconds.
    adjusted_ms = total_ms - avg_diff_ms
    # Converts the adjusted milliseconds back into HH:MM:SS.ms format.
    return ms_to_hhmmssms(adjusted_ms)


# Calls the function to calculate the average time difference between the two logs.
avg_diff = find_time_diff(log_v_path, log_m_path)

# Example: Adjusts a specific MCP timestamp using the calculated average difference.
#mpc_time = "17:17:09.605"  # INPUT MPC_TIME in HH:MM:SS.ms format!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
adjusted_time = subtract_diff_from_time(mpc_time, avg_diff)
print(f"MPC time {mpc_time} minus average difference = Adjusted MPC Time {adjusted_time}")

Average time difference (MCP - ViSaGe): 1534 ms
Average time difference (MM:SS.ms): 00:01.534
MPC time 17:17:09.605 minus average difference = Adjusted MPC Time 17:17:08.071


#### Step 2: Extract Frame Times From Metadata (DeltaT)

#### Read and store acquision time of each frame from image metadata

In [6]:

def extract_deltat_values(file_path_OME):
    """
    Extracts all DeltaT values from a text file, including milliseconds (fractions of seconds).
    
    :param file_path: Path to the input text file.
    :return: List of DeltaT values as floats, including milliseconds.
    """
    # Read the file content
    with open(file_path_OME, 'r') as file:
        content = file.read()
    
    # Find all DeltaT values using regex and convert them to floats
    deltat_values = [float(value) for value in re.findall(r'DeltaT="([0-9.]+)"', content)]
    
    return deltat_values


deltat_values = extract_deltat_values(file_path_OME)

# Print only up to the first 50 values
print(deltat_values[:50])


[0.3380000591278076, 0.5439999103546143, 0.750999927520752, 0.9570000171661377, 1.1640000343322754, 1.369999885559082, 1.5759999752044678, 1.7829999923706055, 1.9890000820159912, 2.196000099182129, 2.4019999504089355, 2.6080000400543213, 2.815000057220459, 3.0209999084472656, 3.2279999256134033, 3.434000015258789, 3.6399998664855957, 3.8469998836517334, 4.052999973297119, 4.259999990463257, 4.466000080108643, 4.671999931335449, 4.878999948501587, 5.085000038146973, 5.29200005531311, 5.497999906539917, 5.703999996185303, 5.91100001335144, 6.116999864578247, 6.323999881744385, 6.5299999713897705, 6.736000061035156, 6.943000078201294, 7.148999929428101, 7.355999946594238, 7.562000036239624, 7.767999887466431, 7.974999904632568, 8.180999994277954, 8.388000011444092, 8.593999862670898, 8.799999952316284, 9.006999969482422, 9.213000059127808, 9.420000076293945, 9.625999927520752, 9.832000017166138, 10.039000034332275, 10.244999885559082, 10.45199990272522]


#### Step 3: Calculate actual time (HH:MM:SS.ms) for each frame on ViSaGe time
> Code uses adjusted MPC acquisition time as time 0 
>> adds the times for each frame as produced in previous code cell 
>>> creates a list of frame times that were aligned with the ViSaGe computer time

In [7]:

# Calculates actual timestamps for each frame using relative DeltaT values and the known first frame time.
def calculate_frame_times(relative_times, first_frame_time):
    """
    Calculate actual timestamps for each frame using DeltaT values and a known first frame time.
    
    :param relative_times: List of relative time points (in seconds).
    :param first_frame_time: The actual time the first frame was collected (as a string in 'HH:MM:SS.sss' format).
    :return: List of datetime objects representing actual timestamps of each frame.
    """
    # Converts the first frame time string into a datetime object.
    first_time = datetime.strptime(first_frame_time, "%H:%M:%S.%f")
    # Adds each relative time (DeltaT) to the first frame time to calculate actual timestamps.
    actual_times = [first_time + timedelta(seconds=t) for t in relative_times]
    return actual_times

# Writes the calculated frame timestamps to a file, numbering each frame.
def write_frame_times_to_file(actual_times, output_file_path_fts):
    """
    Save numbered timestamps to a file.
    
    :param actual_times: List of datetime objects representing actual timestamps of each frame.
    :param output_file_path: Path to the output file where timestamps will be saved.
    """
    # Opens the file for writing and writes each timestamp with its frame number.
    with open(output_file_path_fts, 'w') as file:
        for i, time in enumerate(actual_times):
            # Formats the timestamp to 'HH:MM:SS.sss' and writes it to the file.
            file.write(f"{i+1}: {time.strftime('%H:%M:%S.%f')[:-3]}\n")
    print(f"Frame times successfully written to {output_file_path_fts}")

# Finds the closest frame to a given target time.
def find_closest_frame(actual_times, target_time_str):
    """
    Find the closest frame index to a given time string (HH:MM:SS.sss).
    
    :param actual_times: List of datetime objects representing actual timestamps of each frame.
    :param target_time_str: Target time as a string in 'HH:MM:SS.sss' format.
    :return: Tuple containing the closest frame index (1-based) and its timestamp.
    """
    # Converts the target time string into a datetime object.
    target_time = datetime.strptime(target_time_str, "%H:%M:%S.%f")
    # Finds the index of the frame whose timestamp is closest to the target time.
    closest_index = min(range(len(actual_times)), key=lambda i: abs(actual_times[i] - target_time))
    # Returns the 1-based index and the corresponding timestamp.
    return closest_index + 1, actual_times[closest_index]

# -------------------------------
# USE THIS SECTION TO RUN THE CODE
# -------------------------------   

# The adjusted time from the previous step is used as the first frame time.
first_frame_time = adjusted_time  # This should be the adjusted time from the previous step.

# Assume `deltat_values` is a list of relative time points (in seconds) already defined earlier.
actual_times = calculate_frame_times(deltat_values, first_frame_time)

# Writes the calculated frame times to the specified file.
write_frame_times_to_file(actual_times, output_file_path_fts)



Frame times successfully written to Y:\Jacqui\Critical_Period\Example Analysis\WT_C1_Above\frame_fimes.txt


#### Step 3: Calculate which frame number coincides with the beginning of the ViSaGe video
To do: Enter the ViSaGe time with ms you wrote down when you were imaging 

In [8]:
# Ask user for a time and find the closest frame
user_time = "17:17:18.412" #TIME YOU REORDED IN LAB BOOK!!!!!

try:
    frame_number, timestamp = find_closest_frame(actual_times, user_time)
    print(f"Closest frame to {user_time} is frame #{frame_number} at time {timestamp.strftime('%H:%M:%S.%f')[:-3]}")
except ValueError:
    print("Invalid time format. Please enter the time as HH:MM:SS.sss")

# Print time of frame 50 (index 49 since indexing starts at 0)
#frame_50_time = datetime.strptime(first_frame_time, "%H:%M:%S.%f") + timedelta(seconds=deltat_values[49])
#print(f"Time of frame 50: {frame_50_time.strftime('%H:%M:%S.%f')[:-3]}")

Closest frame to 17:17:18.412 is frame #49 at time 17:17:18.316


### Part 2: Use the user time frame number as offset to generate the epochs in the excel file 

!!! Change to easily input the frame number !!!

In [9]:

# Extracts all DeltaT values from a text file and returns them as a list of floats.
def extract_delta_t_values(file_path_OME):
    """
    Extract all DeltaT values from a text file and return them as a list of floats.
    
    :param file_path: Path to the input text file containing DeltaT values.
    :return: List of DeltaT values as floats.
    """

    delta_t_values = []
    # Regular expression to match DeltaT values in the format DeltaT="value".
    delta_t_pattern = re.compile(r'DeltaT="(\d+\.\d+)"')
    
    # Opens the file and searches for DeltaT values line by line.
    with open(file_path_OME, 'r') as file:
        for line in file:
            match = delta_t_pattern.search(line)
            if match:
                # Converts the matched value to a float and adds it to the list.
                delta_t_values.append(float(match.group(1)))
    
    return delta_t_values

# Creates a new array starting from the ABOVE FOUND DeltaT INDEX VALUE and adding offset values.
def create_new_array(delta_t_values, offset_values):
    """
    Create a new array starting from the frame number calculated above and adding offset values.
    
    :param delta_t_values: List of DeltaT values extracted from the file.
    :param offset_values: List of offset values to add to the initial DeltaT value.
    :return: List of calculated time values.
    """
    
    if len(delta_t_values) < 49: #FRAME NUMBER HERE!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
        # Ensures there are at least 50 DeltaT values in the file.
        raise ValueError("There are fewer than 50 DeltaT values in the file.")
    
    # The 50th value is at index -1 from the number above (0-based indexing).
    initial_value = delta_t_values[48] #FRAME NUMBER HERE (-1)!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    new_array = [initial_value]
    
    # Iteratively adds each offset value to the last value in the array.
    for offset in offset_values:
        new_array.append(new_array[-1] + offset)
    
    return new_array

# Saves the calculated time values to an Excel file with the specified format.
def save_to_excel(time_values, output_file):
    """
    Save the time values to an Excel file with the specified format.
    
    :param time_values: List of calculated time values to save.
    :param output_file: Path to the output Excel file.
    """
    wb = Workbook()
    ws = wb.active
    
    # Write headers to the first row of the Excel file.
    ws['A1'] = "Time (sec)"
    ws['B1'] = "Event Type"
    ws['C1'] = "Event Description"
    
    # Write data to subsequent rows.
    for i, time in enumerate(time_values, 2):  # Start from row 2
        ws.cell(row=i, column=1, value = (time))  # Time values in column A.
        ws.cell(row=i, column=2, value="Marker")  # Event type in column B.
        ws.cell(row=i, column=3, value="trigger")  # Event description in column C.
    
    # Saves the workbook to the specified file path.
    wb.save(output_file)


if __name__ == "__main__":
    # The offset values provided for calculations.
    offset_values = [
        8.16, 10.1, 8.19, 10.11, 8.19, 10.11, 8.17, 10.12,
        8.15, 10.11, 8.19, 10.1, 8.17, 10.1, 8.16, 10.11,
        8.16, 10.11, 8.16, 10.12, 8.18, 10.1, 8.16, 10.12, 8.19
    ]
    
    try:
        # Step 1: Extract all DeltaT values from the input file.
        delta_t_values = extract_delta_t_values(file_path_OME)
        
        # Step 2: Create the new array starting from the 50th DeltaT value.
        time_values = create_new_array(delta_t_values, offset_values)
        
        # Step 3: Save the calculated time values to an Excel file.
        save_to_excel(time_values, output_file)
        print(f"Successfully saved data to {output_file}")
            
    except FileNotFoundError:
        # Handles the case where the input file is not found.
        print(f"Error: The file '{file_path_OME}' was not found.")
    except ValueError as e:
        # Handles errors related to insufficient DeltaT values.
        print(f"Error: {e}")
    except Exception as e:
        # Handles any other unexpected errors.
        print(f"An unexpected error occurred: {e}")

Successfully saved data to Y:\Jacqui\Critical_Period\Example Analysis\WT_C1_Above\Time_Events_for_.csv


## Go back to "Time_Events_for_.csv" and save as UTF-8 .csv and use that in AL analysis